In [ ]:
import os
import numpy as np
from pathlib import Path
import tifffile as tf
from skimage.measure import regionprops
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from napari import Viewer
from functools import reduce
from operator import add
from tqdm import tqdm
from scipy.stats import skewnorm
import seaborn as sns

In [ ]:
path = Path("/home/aaristov/Multicell/Madison/2023-12-06_MLY009_Cas9/pos/Include/pos14/")

In [ ]:
good_labels = pd.read_csv((path / "output" / "table_1.csv"))[["label", "frame"]]
good_labels


In [ ]:
labels = tf.imread((path/"input"/"cellpose.tif"))
labels.shape

In [ ]:
mcherry = tf.imread((path/"input"/"stack.tif"))[:,1]
mcherry.shape

In [ ]:
all_props = reduce(add, [regionprops(lb, intensity_image=img) for lb, img in tqdm(zip(labels, mcherry))])

In [ ]:
good_props = list(filter(lambda p: p.label in good_labels.label.values, all_props))
len(good_props)

In [ ]:
def get_cell_intensities(prop):
    I = prop.image_intensity.ravel()
    cell_intensities = I[I > 0]
    return cell_intensities

In [ ]:
def get_hist_df(prop, step=10):
    try:
        cell_intensities = get_cell_intensities(prop)
        a, loc, scale = skewnorm.fit(cell_intensities)
        h = np.histogram(cell_intensities, bins=range(100,300,step))
    
        return pd.DataFrame({
            "bins": h[1][:-1], 
            "intensities": h[0], 
            "label": prop.label, 
            "area": prop.area, 
            "skew": a, 
            "mod": loc, 
            "mean": prop.mean_intensity,
            "max": prop.max_intensity,
            "ratio": prop.max_intensity / prop.mean_intensity,
            "scale": scale, 
            "pdf": skewnorm.pdf(h[1][:-1], a, loc, scale) * prop.area * step
        })
    except IndexError:
        print("index error: ", prop.label)

In [ ]:
df = pd.concat(map(get_hist_df, tqdm(good_props)), ignore_index=True).merge(good_labels, on="label")


In [ ]:
plot_args = {"height": 500, "hover_data": df.columns}

In [ ]:
px.line(df, x="bins", y="pdf", color="label")

In [ ]:
px.line(df, x="bins", y="intensities", color="label")

In [ ]:
px.scatter(df, x="frame", y="skew", color="max", size="area", **plot_args)

In [ ]:
px.scatter(df, x="frame", y="mod", color="skew", size="area", **plot_args)

In [ ]:
px.scatter(df, x="frame", y="mean", color="skew", size="area",  **plot_args)

In [ ]:
px.scatter(df, x="frame", y="max", color="skew", size="area", **plot_args)

In [ ]:
px.scatter(df, x="frame", y="ratio", color="skew", size="area", **plot_args)

In [ ]:
sns.scatterplot(data=df, x="frame", y="skew")

In [ ]:
sns.lineplot(data=df, x="frame", y="skew")

In [ ]:
sns.heatmap(data=df, 